#A Breakdown of Comp Attendance and Success

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 
import pickle
import numpy as np
import pandas as pd
import tbapy 
from scipy import stats
import DatabaseGen as DBG
from sklearn.decomposition import PCA, KernelPCA
from sklearn.externals import joblib
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, accuracy_score
from sklearn.feature_selection import VarianceThreshold, RFE, SelectKBest, chi2
from sklearn import preprocessing 
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier, AdaBoostClassifier, BaggingRegressor

In [2]:
event_code = "2017alhu"

In [3]:
data = pd.read_pickle("week3" + ".p")

In [4]:
pre_split_pick = data[data['final']==True]
pre_split_npick = data[data['final']==False]
msk = np.random.rand(len(pre_split_npick)) < 0.8
pre_split_npick = pre_split_npick[msk]
data_post = pre_split_npick.append(pre_split_pick)

In [5]:
print(len(pre_split_npick))
print(len(pre_split_pick))

190
192


In [6]:
target = data_post['final'].copy()
data_p = data_post.drop('final', axis=1).copy()
data_p.drop('oprs', axis=1, inplace=True)
data_p.drop('dprs', axis=1, inplace=True)
data_p.drop('ccwms', axis=1, inplace=True)
data_p.drop('rope', axis=1, inplace=True)
data_p.drop('gears', axis=1, inplace=True)
data_p.drop('team_number', axis=1, inplace=True)
#data_p.drop('mean_prev_opr', axis=1, inplace=True)
data_p.drop('team_avg_opr', axis=1, inplace=True)
data_p.drop('oppo_avg_opr', axis=1, inplace=True)

data_p['last_opr_comps'] = stats.rankdata(data_p['last_opr_comps'], "average")/len(data_p)
data_p['opr_comps'] = stats.rankdata(data_p['opr_comps'], "average")/len(data_p)
data_p['mean_prev_opr'] = stats.rankdata(data_p['mean_prev_opr'], "average")/len(data_p)


In [7]:
#msk = np.random.rand(len(data_p)) < 0.8
#data_s = data_p[msk]
#data_r = data_p[~msk]
#target_s = target[msk]
#target_r = target[~msk]
X_train, X_test, y_train, y_test = train_test_split(data_p, target, test_size=0.2, random_state=1337, stratify=target)
#model = RandomForestClassifier(n_estimators=600, min_samples_split=10, max_depth=20)
#model = model.fit(X_train, X_test)
#model.score(data_s, target_s)

In [8]:
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         BaggingRegressor(n_estimators=600))
print(pipeline.get_params().keys())

dict_keys(['memory', 'steps', 'standardscaler', 'baggingregressor', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'baggingregressor__base_estimator', 'baggingregressor__bootstrap', 'baggingregressor__bootstrap_features', 'baggingregressor__max_features', 'baggingregressor__max_samples', 'baggingregressor__n_estimators', 'baggingregressor__n_jobs', 'baggingregressor__oob_score', 'baggingregressor__random_state', 'baggingregressor__verbose', 'baggingregressor__warm_start'])


In [9]:
hyperparameters = {'baggingregressor__max_samples': [1.0, .75, .5, .25], 'baggingregressor__max_features': [1.0, .75, .5, .25], 'baggingregressor__warm_start': [False, True], 'baggingregressor__bootstrap_features': [False, True], 'baggingregressor__bootstrap': [False, True]}

In [10]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10, n_jobs=4)

In [11]:
clf.fit(X_train, y_train)
print(clf.best_params_)

{'baggingregressor__bootstrap': True, 'baggingregressor__bootstrap_features': False, 'baggingregressor__max_features': 1.0, 'baggingregressor__max_samples': 0.25, 'baggingregressor__warm_start': False}


In [12]:
y_pred = clf.predict(X_test)
print(r2_score(y_test, y_pred))
y_pred[y_pred >= y_pred.mean()] = True
y_pred[y_pred != True] = False
print(accuracy_score(y_test, y_pred))

0.296670754316
0.74025974026


In [13]:
clf.score(X_test, y_test)

0.29667075431619772

In [14]:
#data_wrong = data_w_p[data_w_p['final'] != data_w_p['predict']]
#data_w_p['right'] = data_w_p['final'] == data_w_p['predict']

In [15]:
#print(len(data_wrong))
#print(len(data_wrong[data_wrong['final']==True]))

In [16]:
clf.predict([data_p.loc['frc4924']])

array([ 0.02])

In [17]:
#sns.pairplot(X_test, vars=['rookie_year', 'num_comps', 'opr_comps', 'mean_prev_opr'], hue='right', kind="reg")
#plt.show()